In [1]:
import numpy as np
import pandas as pd
import json
import os
import re, requests, bs4, csv, datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',50)
pd.set_option('display.width',1000)

## Scrape CBS Sports

In [2]:
def boxscoreScrapeWk(wkNum):
    print(datetime.datetime.now())
    #Scrape CBS NFL schedule for week "wkNum", and generate boxscore links
    #Scrape each boxscore link from above and write to csv
    #Ex, https://www.cbssports.com/nfl/gametracker/boxscore/NFL_20171210_IND@BUF

    cbs_url = 'https://www.cbssports.com'
    #schedule_url = cbs_url+'/nfl/schedule/2020/regular/'+str(wkNum)
    schedule_url = '/nfl/schedule/2020/regular/'
    
    seasonDF = pd.DataFrame(columns=['Week','Date','Road','Home','url'])
    # Ex, 1, YYYY-MM-DD, AWY, @HME, http://...AWY@HME/
    
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    
    for i in range(1,18):
        weekURL = cbs_url+schedule_url+str(i)
        wkSoup = bs4.BeautifulSoup(session.get(weekURL).text,'html.parser')
        gameURLs = wkSoup.select('.CellGame a[href]')
        numGm = len(gameURLs)
        
        #create DF of one week's games
        thisWkDF = pd.DataFrame(columns=['Week','Date','Road','Home','url'])
        
        for j in range(numGm):
            link = gameURLs[j].get('href')
            p = link.find('NFL_')+4
            thisWkDF.loc[j] = [i] + [str(link[p:p+4]+'-'+link[p+4:p+6]+'-'+link[p+6:p+8])]+[link[p+9:(link.find('@'))]] + [link[(link.find('@')):-1]]+[cbs_url+link] 

        seasonDF = seasonDF.append(thisWkDF)        
        #del(thisWkDF)
        #append above DF to schedDF
        
    gmURLs = seasonDF[seasonDF['Week']==wkNum]['url']
    numGm = len(gmURLs)
    print(numGm)
    #wkSchedSoup = bs4.BeautifulSoup(requests.get(cbs_url+schedule_url+str(wkNum)).text,'html.parser')
    #for 16 matches in i, generate dataframe, game boxscore URLs down one column
    #numGm = len(wkSchedSoup.select('.CellGame a[href]'))
    
    #for i in range(numGm):
    #    gmURLs.loc[i] = cbs_url+str(wkSchedSoup.select('.CellGame a[href]')[i].get('href')).replace('recap','boxscore')

    soupOutput = []
    tagDump = open('nflBoxscoreScrape2020(Tags)_week' + str(wkNum) + '.csv','a',newline='') #change to append
    tagCSVWriter = csv.writer(tagDump,delimiter=',',lineterminator='\n') #change to append

    realOutput = []
    realDump = open('nflBoxscoreScrape2020_week' + str(wkNum) + '.csv','a',newline='')
    realCSVWriter = csv.writer(realDump,delimiter=',',lineterminator='\n')
    
    #######################################################
    ###  NEED SCHEDULE CSV BUT CAN DO FROM PANDA STRAIGHT TO CSV???

    

    
    #schdOutput = []
    #schdDump = open('2020_schedule_single.csv','a',newline='')
    #schdCSVWriter = csv.writer(realDump,delimiter=',',lineterminator='\n')

    seasonDF.drop('url',axis=1).to_csv('2020_schedule_single_week' + str(wkNum) + '.csv', sep = ',', encoding = 'utf-8', index = False)



    #######################################################
    
    tagPick_lines = '.team-stats tr'
    #tagPick_stat_feld/valu = '.team-stats td'
    #print('\n'+str(len(url_game))+'\n')
    #print(url_game[-1])

    # Go through game url list and scrape each url
    for i in range(len(gmURLs)):
        url=str(gmURLs.loc[i])
        boxSoup = bs4.BeautifulSoup(requests.get(url).text,'html.parser')

        # TESTING: Make sure each game has 36 stats(from bs4 Object length) 
        print('Gm '+str(i)+'  '+url+'   '+str(len(boxSoup.select(tagPick_lines))))
        print(datetime.datetime.now())
        p = url.find('NFL_')+4
        soupOutput.append([url[p:-1],boxSoup.select(tagPick_lines)[0].getText()])

        for j in range(1,len(boxSoup.select(tagPick_lines))):
            soupOutput[i].append(boxSoup.select(tagPick_lines)[j].getText())

        tagCSVWriter.writerow(soupOutput[i])

    tagDump.close()
    
    print('NFL Boxscore from CBS Scrape ended.')
    #print('Example row: '+str(soupOutput[0]))   #TO SEE \n 's
    #print('Example row: '+soupOutput[0][35])
    print(datetime.datetime.now())

    # organize soupOutput
    nxt_rcrd = 0
    realOutput.append(['Gm','Date','Team'])
    for h in range(1,len(soupOutput[0])):
        realOutput[nxt_rcrd].append(soupOutput[0][h].splitlines()[0])
        #builds rest of column headings
    nxt_rcrd += 1    

    for i in range(len(soupOutput)):
        url=str(gmURLs.loc[i])
        p = url.find('NFL_')+4    
        realOutput.append([url[p:-1],
                           url[p:p+8],
                           url[url.rfind('_')+1:url.find('@')]  #away team
                           ])
        for j in range(1,len(soupOutput[i])):
            realOutput[nxt_rcrd].append(soupOutput[i][j].splitlines()[1])

        nxt_rcrd += 1    

        realOutput.append([url[p:-1],
                           url[p:p+8],
                           url[url.find('@')+1:len(url)-1]   #home team
                           ])
        for k in range(1,len(soupOutput[i])): #Do something different if j=34,35
                                              #RZ and GoalToGo success rate had
                                              #FIVE lines (\n) to include %.
            if not 33<k<36:
                realOutput[nxt_rcrd].append(soupOutput[i][k].splitlines()[2])
            else:
                realOutput[nxt_rcrd].append(soupOutput[i][k].splitlines()[4])
        nxt_rcrd += 1

    # Organized soupOutput written to csv    
    for i in (range(len(realOutput))):
        realCSVWriter.writerow(realOutput[i])
    realDump.close()

In [3]:
boxscoreScrapeWk(8)

2020-10-27 07:28:24.333347
14
Gm 0  https://www.cbssports.com/nfl/gametracker/live/NFL_20201029_ATL@CAR/   0
2020-10-27 07:28:44.293958


IndexError: list index out of range

## Scrape ESPN Fantasy Football League

In [2]:
swid      = '{F3E3A4D2-1203-4B11-BDAD-00C92AAA3F48}'
espn_s2   = 'AECf2TISP%2BRluNYIj24%2FqkuJBHTzAa7Br0BVIoUnBhHQCygnkN7hvyvU8wIm6XAT58otHnOZW4HjyImLJ14Rk8L9%2BwObWXeIa8kCosNFNMSc79r24KSiJK9jtqUQ2V1lvIncFM1qDhV9xL7E5jnutCP9rZfiJv8h%2Bq6WNvFb4YmyHICgx3QW68obk3wqBrGrXmw0vbq2bf367%2BsuL%2BOJ2ioRnxi1yY7LgbtiJnAXSKZj7kdIGjBiCPjUHgeXzDLHPunkLchEGlOrr%2FhG1ORgOsUX'
league_id = 169073
season    = 2020
week      = 16

url = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/' + \
      str(season) + '/segments/0/leagues/' + str(league_id) + \
      '?view=mMatchup&view=mMatchupScore'

print(url)

r = requests.get(url,
                 params={'scoringPeriodId': week},
                 cookies={"SWID": swid, "espn_s2": espn_s2})
d = r.json()

https://fantasy.espn.com/apis/v3/games/ffl/seasons/2020/segments/0/leagues/169073?view=mMatchup&view=mMatchupScore


In [3]:
import json
with open('matchups_week_16.json', 'w', encoding='utf-8') as f:
    json.dump(d, f, ensure_ascii=False, indent=4)

In [ ]:
#swid      = '{F3E3A4D2-1203-4B11-BDAD-00C92AAA3F48}'
espn_s2   = 'AECf2TISP%2BRluNYIj24%2FqkuJBHTzAa7Br0BVIoUnBhHQCygnkN7hvyvU8wIm6XAT58otHnOZW4HjyImLJ14Rk8L9%2BwObWXeIa8kCosNFNMSc79r24KSiJK9jtqUQ2V1lvIncFM1qDhV9xL7E5jnutCP9rZfiJv8h%2Bq6WNvFb4YmyHICgx3QW68obk3wqBrGrXmw0vbq2bf367%2BsuL%2BOJ2ioRnxi1yY7LgbtiJnAXSKZj7kdIGjBiCPjUHgeXzDLHPunkLchEGlOrr%2FhG1ORgOsUX'
#league_id = 169073
#season    = 2020
#week      = 1


s = requests.Session()
r = s.get('https://www.espn.com')

swid = s.cookies.get_dict()['SWID']


league_id = 169073


url = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/2020/segments/0/leagues/%s' %league_id


r = requests.get(url, cookies={"swid": swid, "espn_s2": espn_s2}).json()

#Get Team IDs
teamId = {}
for team in r['teams']:
    teamId[team['id']] = team['location'].strip() + ' ' + team['nickname'].strip()


#Get each team's weekly points and calculate their head-to-head records
weeklyPoints = {}
r = requests.get(url, cookies={"swid": swid}, params={"view": "mMatchup"}).json()

weeklyPts = pd.DataFrame()
for each in r['schedule']:
    #each = r['schedule'][0]

    week = each['matchupPeriodId']
    if week >= 14:
        continue

    homeTm = teamId[each['home']['teamId']]
    homeTmPts = each['home']['totalPoints']

    try:
        awayTm = teamId[each['away']['teamId']]
        awayTmPts = each['away']['totalPoints']
    except:
        homeTmPts = 'BYE'
        continue

    temp_df = pd.DataFrame(list(zip([homeTm, awayTm], [homeTmPts, awayTmPts], [week, week])), columns=['team','pts','week'])

    if homeTmPts > awayTmPts:
        temp_df.loc[0,'win'] = 1
        temp_df.loc[0,'loss'] = 0
        temp_df.loc[0,'tie'] = 0

        temp_df.loc[1,'win'] = 0
        temp_df.loc[1,'loss'] = 1
        temp_df.loc[1,'tie'] = 0

    elif homeTmPts < awayTmPts:
        temp_df.loc[0,'win'] = 0
        temp_df.loc[0,'loss'] = 1
        temp_df.loc[0,'tie'] = 0

        temp_df.loc[1,'win'] = 1
        temp_df.loc[1,'loss'] = 0
        temp_df.loc[1,'tie'] = 0

    elif homeTmPts == awayTmPts:
        temp_df.loc[0,'win'] = 0
        temp_df.loc[0,'loss'] = 0
        temp_df.loc[0,'tie'] = 1

        temp_df.loc[1,'win'] = 0
        temp_df.loc[1,'loss'] = 0
        temp_df.loc[1,'tie'] = 1

    weeklyPts = weeklyPts.append(temp_df, sort=True).reset_index(drop=True)

weeklyPts['win'] = weeklyPts.groupby(['team'])['win'].cumsum()
weeklyPts['loss'] = weeklyPts.groupby(['team'])['loss'].cumsum()
weeklyPts['tie'] = weeklyPts.groupby(['team'])['tie'].cumsum()



# Calculate each teams record compared to all other teams points week to week
cumWeeklyRecord = {}   
for week in weeklyPts[weeklyPts['pts'] > 0]['week'].unique():
    df = weeklyPts[weeklyPts['week'] == week]

    cumWeeklyRecord[week] = {}
    for idx, row in df.iterrows():
        team = row['team']
        pts = row['pts']
        win = len(df[df['pts'] < pts])
        loss = len(df[df['pts'] > pts])
        tie = len(df[df['pts'] == pts])

        cumWeeklyRecord[week][team] = {}
        cumWeeklyRecord[week][team]['win'] = win
        cumWeeklyRecord[week][team]['loss'] = loss
        cumWeeklyRecord[week][team]['tie'] = tie-1

# Combine those cumluative records to get an overall season record      
overallRecord = {}     
for each in cumWeeklyRecord.items():
    for team in each[1].keys():
        if team not in overallRecord.keys():
            overallRecord[team] = {} 

        win = each[1][team]['win']
        loss = each[1][team]['loss']
        tie = each[1][team]['tie']

        if 'win' not in overallRecord[team].keys():
            overallRecord[team]['win'] = win
        else:
            overallRecord[team]['win'] += win

        if 'loss' not in overallRecord[team].keys():
            overallRecord[team]['loss'] = loss
        else:
            overallRecord[team]['loss'] += loss

        if 'tie' not in overallRecord[team].keys():
            overallRecord[team]['tie'] = tie
        else:
            overallRecord[team]['tie'] += tie


# Little cleaning up of the data nd calculating win %
overallRecord_df = pd.DataFrame(overallRecord).T
overallRecord_df = overallRecord_df.rename_axis('team').reset_index()
overallRecord_df = overallRecord_df.rename(columns={'win':'overall_win', 'loss':'overall_loss','tie':'overall_tie'})
overallRecord_df['overall_win%'] = overallRecord_df['overall_win'] / (overallRecord_df['overall_win'] + overallRecord_df['overall_loss'] + overallRecord_df['overall_tie'])
overallRecord_df['overall_rank'] = overallRecord_df['overall_win%'].rank(ascending=False, method='min')




regularSeasRecord = weeklyPts[weeklyPts['week'] == 13][['team','win','loss', 'tie']]
regularSeasRecord['win%'] = regularSeasRecord['win'] / (regularSeasRecord['win'] + regularSeasRecord['loss'] + regularSeasRecord['tie'])
regularSeasRecord['rank'] = regularSeasRecord['win%'].rank(ascending=False, method='min')



final_df = overallRecord_df.merge(regularSeasRecord, how='left', on=['team'])

In [ ]:
#swid      = '{F3E3A4D2-1203-4B11-BDAD-00C92AAA3F48}'
espn_s2   = 'AECf2TISP%2BRluNYIj24%2FqkuJBHTzAa7Br0BVIoUnBhHQCygnkN7hvyvU8wIm6XAT58otHnOZW4HjyImLJ14Rk8L9%2BwObWXeIa8kCosNFNMSc79r24KSiJK9jtqUQ2V1lvIncFM1qDhV9xL7E5jnutCP9rZfiJv8h%2Bq6WNvFb4YmyHICgx3QW68obk3wqBrGrXmw0vbq2bf367%2BsuL%2BOJ2ioRnxi1yY7LgbtiJnAXSKZj7kdIGjBiCPjUHgeXzDLHPunkLchEGlOrr%2FhG1ORgOsUX'
#league_id = 169073
#season    = 2020
#week      = 1


s = requests.Session()
r = s.get('https://www.espn.com')

swid = s.cookies.get_dict()['SWID']


league_id = 169073


url = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/2020/segments/0/leagues/%s' %league_id


r = requests.get(url, cookies={"swid": swid, "espn_s2": espn_s2}).json()

#Get Team IDs
teamId = {}
for team in r['teams']:
    teamId[team['id']] = team['location'].strip() + ' ' + team['nickname'].strip()

teamId
#Get each team's weekly points and calculate their head-to-head records
weeklyPoints = {}
r = requests.get(url, cookies={"swid": swid}, params={"view": "mMatchup"}).json()

In [ ]:
import json
with open('second_json_attempt.json', 'w', encoding='utf-8') as f:
    json.dump(r, f, ensure_ascii=False, indent=4)

In [ ]:
# Opening JSON file 
f = open('third_json_attempt.json') 
  
# returns JSON object as  
# a dictionary 
data = json.load(f) 

In [ ]:
print(data['teams'][0]['roster']['entries'][0]['playerPoolEntry']['player']['stats'][5]['statSourceId'])
print(data['teams'][0]['roster']['entries'][0]['playerPoolEntry']['player']['stats'][5]['appliedTotal'])

In [ ]:
print(data['teams'][0]['roster']['entries'][0]['playerPoolEntry']['player']['stats'][2]['statSourceId'])
print(data['teams'][0]['roster']['entries'][0]['playerPoolEntry']['player']['stats'][2]['appliedTotal'])

In [ ]:
data.keys()

In [ ]:
for i in range(0, 10):
    print(data['schedule'][i]['id'])
    #print(i.keys())

In [ ]:
for i in data['schedule']:
    print(i.keys())

In [ ]:
print("active player: ", data['schedule'][0]['away']['rosterForMatchupPeriod']['entries'][0]['playerPoolEntry']['player']['fullName'])
print("actual points:", data['schedule'][0]['away']['rosterForMatchupPeriod']['entries'][0]['playerPoolEntry']['appliedStatTotal'])

In [ ]:
print("player:", data['teams'][0]['roster']['entries'][0]['playerPoolEntry']['player']['fullName'])
print("points:", data['teams'][0]['roster']['entries'][0]['playerPoolEntry']['appliedStatTotal'])
print("current injury status:", data['teams'][0]['roster']['entries'][0]['playerPoolEntry']['player']['injuryStatus'])
print("team_id:", data['teams'][0]['roster']['entries'][0]['playerPoolEntry']['player']['proTeamId'])
print("position_id:", data['teams'][0]['roster']['entries'][0]['playerPoolEntry']['player']['defaultPositionId'])

In [ ]:
len(data['teams'][9]['roster']['entries'])

In [4]:
team_id = []
player = []
points = []
current_injury_status = []
matchup_injury_status = []
lineup_slot_id = []
position_id = []
pro_team_id = []

for i in range(0, len(data['teams'])):
    for j in range(0, len(data['teams'][i]['roster']['entries'])):
        team_id.append(data['teams'][i]['id'])
        player.append(data['teams'][i]['roster']['entries'][j]['playerPoolEntry']['player']['fullName'])
        points.append(data['teams'][i]['roster']['entries'][j]['playerPoolEntry']['appliedStatTotal'])
        #current_injury_status.append(data['teams'][i]['roster']['entries'][j]['playerPoolEntry']['player']['injuryStatus'])
        matchup_injury_status.append(data['teams'][i]['roster']['entries'][j]['injuryStatus'])
        lineup_slot_id.append(data['teams'][i]['roster']['entries'][j]['lineupSlotId'])
        position_id.append(data['teams'][i]['roster']['entries'][j]['playerPoolEntry']['player']['defaultPositionId'])
        pro_team_id.append(data['teams'][i]['roster']['entries'][j]['playerPoolEntry']['player']['proTeamId'])
        
rosters_df = pd.DataFrame({'team_id' : team_id, 
                           'player' : player, 
                           'points' : points, 
                           'matchup_injury_status': matchup_injury_status,
                           #'current_injury_status' : current_injury_status,
                           'lineup_slot_id': lineup_slot_id,
                           'position_id' : position_id,
                           'pro_team_id' : pro_team_id
                          })

rosters_df.loc[rosters_df['team_id'] == 1].sort_values('lineup_slot_id')

NameError: name 'data' is not defined

In [ ]:
rosters_df['pro_team_id'].value_counts().sort_index()

In [ ]:
rosters_df.loc[rosters_df['pro_team_id'] == 0]

In [ ]:
away_team_id = []
away_team_score = []
home_team_id = []
home_team_score = []
matchupPeriod = []
winner = []

week = []
team_id = []
opp_id = []
score = []
win = []


for i in range(0, len(data['schedule'])):
    
    if data['schedule'][i]['away']['totalPoints'] == 0:
        break
        
    # build row for away team
    week.append(data['schedule'][i]['matchupPeriodId'])
    team_id.append(data['schedule'][i]['away']['teamId'])
    score.append(data['schedule'][i]['away']['totalPoints'])
    opp_id.append(data['schedule'][i]['home']['teamId'])
    
    # determine if away team won
    if data['schedule'][i]['winner'] == 'AWAY':
        win.append(1)
    else:
        win.append(0)
    
    # build row for home team
    week.append(data['schedule'][i]['matchupPeriodId'])
    team_id.append(data['schedule'][i]['home']['teamId'])
    score.append(data['schedule'][i]['home']['totalPoints'])
    opp_id.append(data['schedule'][i]['away']['teamId'])
    
    # determine if home team won    
    if data['schedule'][i]['winner'] == 'HOME':
        win.append(1)
    else:
        win.append(0)
    
df = pd.DataFrame({'week': week, 'team_id': team_id, 'opp_id': opp_id, 'score': score, 'win': win})

df

In [ ]:
g = sns.lineplot(data=df, x="week", y="score", hue = 'team_id')
g.legend(loc='center right', bbox_to_anchor=(1.25, 0.5), ncol=1)
plt.show()

In [ ]:
pip install plotly==4.11.0

In [ ]:
fig = px.line(df, x="week", y="score", color = 'team_id', title='Score per Week')
fig.update_xaxes(range=[0.95, 5.05], dtick=1)
fig.show()

In [ ]:
plt.plot(df['week'].loc[(df['team_id'] == 6)], df['score'].loc[(df['team_id'] == 6)], label = 6)
plt.plot(df['week'].loc[(df['team_id'] == 1)], df['score'].loc[(df['team_id'] == 1)], label = 1)
plt.plot(df['week'].loc[(df['team_id'] == 2)], df['score'].loc[(df['team_id'] == 2)], label = 2)
plt.plot(df['week'].loc[(df['team_id'] == 15)], df['score'].loc[(df['team_id'] == 15)], label = 15)
plt.plot(df['week'].loc[(df['team_id'] == 7)], df['score'].loc[(df['team_id'] == 7)], label = 7)
plt.plot(df['week'].loc[(df['team_id'] == 11)], df['score'].loc[(df['team_id'] == 11)], label = 11)
plt.plot(df['week'].loc[(df['team_id'] == 4)], df['score'].loc[(df['team_id'] == 4)], label = 4)
plt.plot(df['week'].loc[(df['team_id'] == 8)], df['score'].loc[(df['team_id'] == 8)], label = 8)
plt.plot(df['week'].loc[(df['team_id'] == 9)], df['score'].loc[(df['team_id'] == 9)], label = 9)
plt.plot(df['week'].loc[(df['team_id'] == 10)], df['score'].loc[(df['team_id'] == 10)], label = 10)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
df['team_id'].unique()

In [ ]:
df['week'].loc[(df['team_id'] == 6)]

In [ ]:
# Opening JSON file 
f = open('second_json_attempt.json') 
  
# returns JSON object as  
# a dictionary 
data = json.load(f) 
data.keys()

In [ ]:
len(data)

## Process FF File in weekly Directory

In [ ]:
file_list = []

for directory in os.listdir('weekly'):
    
    for sub_directory in os.listdir('weekly' + '/' + directory):
        
        file_list.append('weekly' + '/' + directory + '/' + sub_directory)

In [ ]:
def read_ff_csv(filenames):
    
    df = pd.DataFrame()
    
    for filename in filenames:
        print(filename)
        dummy_df = pd.read_csv(filename)
        dummy_df['Year'] = int(re.search(r'/(.+?)/', filename).group(1))
        dummy_df['Week'] = int(re.search(r'\/week(.+?)\.', filename).group(1))
        df = df.append(dummy_df, ignore_index = True)
    
    df = df[['Player', 'Pos', 'Tm', 'Year', 'Week', 'PassingYds', 'PassingTD', 'Int', 'PassingAtt', 'Cmp', 'RushingAtt', 'RushingYds', 
             'RushingTD', 'Rec', 'Tgt', 'ReceivingYds', 'ReceivingTD', 'FL', 'PPRFantasyPoints', 'StandardFantasyPoints', 
             'HalfPPRFantasyPoints']].sort_values(by = ['Year', 'Week'])
    
    return df

In [ ]:
ff_df = read_ff_csv(file_list)
ff_df.shape